# Shuffle and learn

Train encoder to predict if 3 frames are in correct temporal order or not.  
Paper: https://arxiv.org/abs/1603.08561

In [1]:
import cv2

import sys

sys.path.append('/scratch/mz2476/DL/project/')

import os
import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200
from ssl_project.data_loaders import plot_utils

import imageio

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from ssl_project.data_loaders.data_helper import UnlabeledDataset, LabeledDataset
from ssl_project.data_loaders.helper import collate_fn, draw_box
from ssl_project import constants

from ssl_project.preprocessing import top_down_segmentation


from ssl_project.utils import to_np

In [2]:
from ssl_project.constants import *
from ssl_project.paths import *

In [3]:
from ssl_project.ssl_ideas.preprocessing import TripleDataset

# Load the data:
triples of photos, can be separate datasets for each camera view

In [4]:
# for idx in range(5):
#     img = imageio.imread(f"../data/scene_30/sample_{idx}/CAM_FRONT.jpeg")
#     plt.imshow(img)
#     plt.show()

In [5]:
# train_dataset = TripleDataset(cam_names=["CAM_FRONT"], scene_ids=UNLABELED_SCENE_INDEX)
# len(train_dataset)

## Explore

In [6]:
# positive_counter = 0 
# for idx in range(0, 1000, 10):
#     if positive_counter > 10: 
#         break 
#     images_o3hw, label = train_dataset[idx]
    
#     if label == 1:
#         positive_counter += 1
#         plot_utils.plot_photos(images_o3hw)
        
        

In [7]:
# positive_counter = 0 # negative ...
# for idx in range(0, 1000, 10):
#     if positive_counter > 10: 
#         break 
#     images_o3hw, label = train_dataset[idx]
    
#     if label == 0:
#         positive_counter += 1
#         plot_utils.plot_photos(images_o3hw)

# Train model

In [8]:
import pytorch_lightning as pl
from ssl_project.road_layout_prediction.modelzoo import encoder
from torch.utils.data import DataLoader

In [9]:
# matplotlib.rcParams['figure.figsize'] = [5, 5]
# matplotlib.rcParams['figure.dpi'] = 200

seed = 8964

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed);

# torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.deterministic = True

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [10]:
# model = torchvision.models.resnet18(pretrained=True)
# print(model)

In [11]:
# encoder()

In [12]:
from ssl_project.constants import LABELED_SCENE_INDEX, UNLABELED_SCENE_INDEX

In [13]:
train_idces = UNLABELED_SCENE_INDEX
val_idces = LABELED_SCENE_INDEX

In [19]:
class ShuffleAndLearnNet(nn.Module):
    def __init__(self, resnet_style='18', pretrained=False):
        super().__init__()
        # compute hparams
        self.hparams = {
            key : value for key, value in locals().items()
            if not (key == "self" or key.startswith("__"))
        }
        
        self.resnet_encoder = encoder(resnet_style='18', pretrained=False)
        OUT_ENC_CHANNELS = 512 # might change in the future
        # output 512 x 8 x 8
        self.decoder = nn.Sequential(
            nn.Conv2d(3 * OUT_ENC_CHANNELS, OUT_ENC_CHANNELS, kernel_size=(3, 3), stride=(1, 1), bias=False),
            nn.BatchNorm2d(OUT_ENC_CHANNELS, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(output_size=(1, 1)),
        )
        self.clf = nn.Linear(in_features=OUT_ENC_CHANNELS, out_features=2, bias=True)
        
    def forward(self, images_bo3hw):
        # c = 512, l = 8
        image_1_bcll = self.resnet_encoder(images_bo3hw[:, 0])
        image_2_bcll = self.resnet_encoder(images_bo3hw[:, 1])
        image_3_bcll = self.resnet_encoder(images_bo3hw[:, 2])
        out_bm11 = self.decoder(torch.cat((image_1_bcll, image_2_bcll, image_3_bcll), dim=1))
#         assert out_bm11.shape[-2:] == (1, 1)
        out_b2 = self.clf(out_bm11.view(out_bm11.shape[:2]))
        return out_b2

In [20]:
class ShuffleAndLearnModel(pl.LightningModule):
    def __init__(self, model, hparams):
        super().__init__()
        self.model = model
        self.cam_names = hparams["cam_names"]
        self.hparams = hparams
        self.hparams["cam_names"] = "__".join(self.hparams["cam_names"])
#         assert len(set(hparams.keys()) & set(self.model.hparams.keys())) == 0
        self.hparams.update({f"model_{k}" : v for k, v in self.model.hparams.items()})
        
        self.criterion = nn.BCEWithLogitsLoss(reduction="sum")
        self.threshold = 0.5
        self.num_workers = 4
        
#     def forward(self, images_bo3hw, cam_name):
#         resnet_encoder = self.cam_name_to_encoder[cam_name]
#         image_1_bcll = resnet_encoder(image_bo3hw[:, 0])
#         image_2_bcll = resnet_encoder(image_bo3hw[:, 1])
#         image_3_bcll = resnet_encoder(image_bo3hw[:, 2])
        
#         out_b2 = self.classifier(torch.cat((image_1_bcll, image_2_bcll, image_3_bcll), dim=1))
#         return out_b2
    
        
    def forward(self, images_bo3hw):
        return self.model(images_bo3hw)
    
    def training_step(self, batch, batch_idx):
        images_bo3hw, labels_b = batch
        out_b = self.forward(images_bo3hw)[:, 1]
        loss = self.criterion(out_b, labels_b.type_as(out_b))
        tensorboard_logs = {'train_loss': loss}
        return {
            'loss': loss, 
            'log': tensorboard_logs
        }
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        return DataLoader(
            TripleDataset(cam_names=self.cam_names, scene_ids=train_idces), 
            num_workers=self.num_workers, 
            batch_size=32,
            shuffle=True,
            pin_memory=True,
        )

    def val_dataloader(self):
        return DataLoader(
            TripleDataset(cam_names=self.cam_names, scene_ids=val_idces), 
            num_workers=self.num_workers, 
            batch_size=32, 
            shuffle=False, 
            pin_memory=True
        )
    
    
    def on_train_start(self):
        self.logger.log_hyperparams_metrics(
            self.hparams, {'val_loss': 1, 'val_acc': 0})

    
    def validation_step(self, batch, batch_idx):
        images_bo3hw, labels_b = batch
        out_b = self.forward(images_bo3hw)[:, 1]
        return {
            'val_loss': self.criterion(out_b, labels_b.type_as(out_b)),
            'val_acc' : ((out_b > self.threshold).long() == labels_b).float().sum(),
        }
    
    def validation_epoch_end(self, outputs):
        avg_loss = (torch.stack([x['val_loss'] for x in outputs]).sum() / self.val_size).item()
        avg_acc = (torch.stack([x['val_acc'] for x in outputs]).sum() / self.val_size).item()

        tensorboard_logs = {'val_loss': avg_loss, 'val_acc' : avg_acc,}
                
        return {
            'val_loss': avg_loss, 
            'val_acc' : avg_acc,
            'log': tensorboard_logs
        }
 
    @property
    def val_size(self):
        return len(self.val_dataloader().dataset)



In [21]:
# from torch.utils.tensorboard import SummaryWriter

# trainer.logger.log_dir

In [36]:
from logger_hparams import HyperparamsSummaryTensorBoardLogger

logger = HyperparamsSummaryTensorBoardLogger("lightning_logs", name="cam_front_full", version="02")


net = ShuffleAndLearnNet()
model_with_data = ShuffleAndLearnModel(net, {"cam_names" : ["CAM_FRONT"]})

trainer = pl.Trainer(gpus=1, 
#                      auto_lr_find=True, 
                     show_progress_bar=True,
#                      train_percent_check=0.02,
#                      val_percent_check=0.05,
                     logger=logger,
            )



INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


In [29]:
trainer.logger

model_with_data.logger = trainer.logger

trainer.logger

model_with_data.logger

In [ ]:
trainer.fit(model_with_data, )

INFO:lightning:Set SLURM handle signals.
INFO:lightning:
   | Name                                                 | Type               | Params
----------------------------------------------------------------------------------------
0  | model                                                | ShuffleAndLearnNet | 18 M  
1  | model.resnet_encoder                                 | encoder            | 11 M  
2  | model.resnet_encoder.resnet_encoder                  | Sequential         | 11 M  
3  | model.resnet_encoder.resnet_encoder.0                | Conv2d             | 9 K   
4  | model.resnet_encoder.resnet_encoder.1                | BatchNorm2d        | 128   
5  | model.resnet_encoder.resnet_encoder.2                | ReLU               | 0     
6  | model.resnet_encoder.resnet_encoder.3                | MaxPool2d          | 0     
7  | model.resnet_encoder.resnet_encoder.4                | Sequential         | 147 K 
8  | model.resnet_encoder.resnet_encoder.4.0              | Ba

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [41]:
model_with_data.hparams

{'asbsadf': 234,
 'model_a': 2,
 'model_resnet_style': '18',
 'model_pretrained': False}

In [18]:
net = ShuffleAndLearnNet()
model_with_data = ShuffleAndLearnModel(net)
trainer = pl.Trainer(gpus=1)    
trainer.fit(model)  

In [19]:
model_with_data.hparams

In [ ]:
trainer = pl.Trainer(gpus=1)    

In [ ]:
model = ShuffleAndLearnModel()
trainer = pl.Trainer(gpus=1)    
trainer.fit(model)  

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:Set SLURM handle signals.
INFO:lightning:
   | Name                                           | Type              | Params
---------------------------------------------------------------------------------
0  | resnet_encoder                                 | encoder           | 11 M  
1  | resnet_encoder.resnet_encoder                  | Sequential        | 11 M  
2  | resnet_encoder.resnet_encoder.0                | Conv2d            | 9 K   
3  | resnet_encoder.resnet_encoder.1                | BatchNorm2d       | 128   
4  | resnet_encoder.resnet_encoder.2                | ReLU              | 0     
5  | resnet_encoder.resnet_encoder.3                | MaxPool2d         | 0     
6  | resnet_encoder.resnet_encoder.4                | Sequential        | 147 K 
7  | resnet_encoder.resnet_encoder.4.0              | BasicBlock        | 73 K  
8  | resnet_encoder.resnet_encoder.4.0.conv1

In [32]:
trainer.__dict__

# save best model weights
# tensorboard
# print log loss

{'callbacks': [<pytorch_lightning.callbacks.progress.ProgressBar at 0x2ac2ae1a15e0>],
 'benchmark': False,
 'num_nodes': 1,
 'log_gpu_memory': None,
 'gradient_clip_val': 0,
 'check_val_every_n_epoch': 1,
 'track_grad_norm': -1,
 'on_gpu': True,
 'on_tpu': False,
 'num_tpu_cores': None,
 'num_processes': 1,
 'process_position': 0,
 'weights_summary': 'full',
 'max_epochs': 1000,
 'min_epochs': 1,
 'max_steps': None,
 'min_steps': None,
 'num_sanity_val_steps': 5,
 'reload_dataloaders_every_epoch': False,
 'auto_lr_find': False,
 'replace_sampler_ddp': True,
 'truncated_bptt_steps': None,
 'resume_from_checkpoint': None,
 'terminate_on_nan': False,
 'shown_warnings': set(),
 'fast_dev_run': False,
 'default_root_dir': '/scratch/mz2476/DL/project/ssl_project/ssl_ideas',
 'total_batch_idx': 833,
 'running_loss': <pytorch_lightning.trainer.supporters.TensorRunningAccum at 0x2ac2aeff3d00>,
 'batch_idx': 138,
 'progress_bar_metrics': {},
 'callback_metrics': {'loss': tensor(9.6165, device='c

In [34]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Launching TensorBoard...

AttributeError: module 'cgi' has no attribute 'escape'

In [25]:
%debug

> <ipython-input-21-1ac3f00228d1>(1)<module>()
----> 1 trainer.val_dataloader()

ipdb> up
*** Oldest frame
ipdb> q
